# Amazon Bedrock AgentCore Gateway - Semantic search tutorial

### 튜토리얼 세부 정보


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| AgentCore services  | AgentCore Gateway, AgentCore Identity                                            |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                        |
| Tutorial components | Creating and using Lambda-backed AgentCore Gateway from Strands Agent            |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3  

### 튜토리얼 아키텍처
Amazon Bedrock AgentCore Gateway provides unified connectivity between agents and the tools and resources they need to interact with. Gateway plays multiple roles in this connectivity layer:

1. **Security Guard**: Gateway manages OAuth authorization to ensure only valid users / agents access tools / resources.
    "2. **번역기**: Gateway는 Model Context Protocol(MCP)과 같은 인기 있는 프로토콜을 사용하여 만든 에이전트 요청을 API 요청과 Lambda 호출로 번역합니다. 이는 개발자가 서버를 호스팅하거나 프로토콜 통합, 버전 지원, 버전 패치 등을 관리할 필요가 없음을 의미합니다.\n",
3. **Composer**: Gateway enables developers to seamlessly combine multiple APIs, functions, and tools into a single MCP  endpoint that an agent can use.
4. **Keychain**: Gateway handles the injection of the right credentials to use with the right tool, ensuring that agents can seamlessly leverage tools that require different sets of credentials. 
    "5. **연구자**: Gateway는 에이전트가 모든 도구를 검색하여 주어진 컨텍스트나 질문에 가장 적합한 도구만 찾을 수 있도록 합니다. 이를 통해 에이전트는 소수의 도구가 아닌 수천 개의 도구를 활용할 수 있습니다. 또한 에이전트의 LLM 프롬프트에 제공해야 하는 도구 세트를 최소화하여 지연 시간과 비용을 줄입니다. \n",
6. **Infrastructure Manager**: Gateway is completely serverless, and comes with built-in observability and auditing, alleviating the need for developers to manage additional infrastructure to integrate their agents and tools. 

![작동 방식](images/gw-arch-overview.png)

### Tutorial Key Features

* Creating Amazon Bedrock AgentCore Gateways with AWS Lambda-backed targets
* Using AgentCore Gateway semantic search 
* Using Strands Agents to show how AgentCore Gateway search improves latency

## 전제 조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway는 많은 수의 도구를 가진 MCP 서버의 문제를 해결하는 데 도움이 됩니다
일반적인 엔터프라이즈 환경에서 에이전트 빌더는 수백 또는 수천 개의
MCP 도구를 가진 MCP 서버를 접하게 됩니다. 이러한 도구의 양은 도구 선택 정확도 저하, 
비용 증가, 과도한 도구 메타데이터로 인한 높은 토큰 사용량으로 인한 지연 시간 증가 등 AI 에이전트에게 도전 과제를 제기합니다.
이는 에이전트를 타사 서비스(예: Zendesk, Salesforce,Slack, JIRA, ...) 또는 기존 엔터프라이즈 REST 서비스에 연결할 때 발생할 수 있습니다. 
AgentCore Gateway provides a built in semantic search across tools, 
which improves agent latency, cost, and accuracy, while still giving those agents the tools they need. 
Depending on your use case, LLM model, and agent framework, you can see up to 3x better latency by keeping
an agent focused on relevant tools versus providing the full set of hundreds of tools from a typical MCP Server.

![작동 방식](images/gateway_tool_search.png)

## 이 노트북에서 배울 내용이 노트북에서는 AgentCore Gateway 검색에 대한 튜토리얼을 제공합니다. 이 단계별 튜토리얼을 마치면 다음을 할 수 있습니다:이해할 내용:

- AgentCore Gateway의 내장 검색 도구를 사용하여 관련 도구를 빠르게 찾는 방법- 지연 시간 개선과 비용 절감을 위해 도구 검색 결과를 Strands Agents에 통합하는 방법
## 개요 of the notebook structure
노트북은 다음 섹션으로 구성되어 있습니다:
1. Understanding fundamentals of AgentCore Gateway Search
2. Preparing the notebook environment
3. Setting up a Gateway that has hundreds of tools
4. Searching for tools from a Gateway
5. Using Strands Agents with an MCP server that has many tools
6. Adding tool search results to a Strands Agent
7. Showing 3x latency improvement by using tool search

# Understanding fundamentals of AgentCore Gateway Search

AgentCore Gateway를 생성할 때 검색을 활성화하려는 옵션을 지정할 수 있습니다.검색이 활성화된 Gateway의 경우 다음 세 가지가 발생합니다:
1. **벡터 저장소가 생성됩니다**. Gateway 서비스는 새 Gateway를 위해 서버리스 완전 관리형 벡터 저장소를 자동으로 생성합니다. 이를 통해 Gateway 도구 전반에 걸친 완전한 의미 검색이 가능합니다. 
3. **벡터 저장소가 채워집니다**. Gateway에 Gateway Target을 추가하면 서비스가 자동으로 백그라운드에서 임베딩을 사용하여 새 Target의 도구를 기반으로 벡터 저장소를 채웁니다. 도구 메타데이터는 도구의 JSON 정의 또는 REST 서비스 대상의 OpenAPI 스키마 사양에서 가져옵니다.
2. **검색 도구(MCP 기반)가 제공됩니다**. 모든 사용자 정의 도구(AWS Lambda 대상 또는 REST 서비스에서)에 추가로 Gateway는 의미 검색을 제공하는 하나의 추가 MCP 도구를 얻습니다. 이름은 `x-amz-bedrock-agentcore-search`입니다. 접두사는 사용자 정의 도구와 이름 충돌이 없도록 보장합니다. 향후에도 이와 같은 도구를 더 추가할 수 있습니다. 검색 도구에는 `query`라는 단일 인수가 있습니다. 검색 도구가 호출되면 Gateway 서비스는 해당 쿼리를 사용하여 의미 검색을 수행하고, 사용 가능한 도구 메타데이터(이름, 설명, 입력 및 출력 스키마)와 일치시켜 관련성 내림차순으로 가장 관련성이 높은 도구를 반환합니다.

# Preparing the notebook environment

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

Import all the required Python libraries, and load environment variables

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

Set up a logger

In [ ]:
# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

# 로그를 보기 위한 핸들러 추가logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

Check our boto3 version

In [ ]:
boto3.__version__

Get our boto3 client for the AgentCore control plane API.

In [ ]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
)

# Setting up a Gateway that has hundreds of tools

AgentCore Gateway provides a secure and scalable way to expose a curated set of existing APIs
에이전트용 MCP 도구로 사용됩니다. 프로덕션 환경에서는 Gateway 리소스가 생성됩니다CloudFormation, CDK 또는 Terraform과 같은 도구를 사용하여 코드형 인프라를 사용합니다. 이 튜토리얼에서는
we use the boto3 control plane APIs directly, so that you can understand the resources and APIs more effectively.
이를 통해 자신만의 게이트웨이를 구축하고 사용하며, 더 강력한
and secure agents.

At a high level, the steps for setting up your Gateway are:

1. Define what identity providers and credential providers you are using for inbound (agents calling Gateways) and outbound (Gateways calling tools) security.
2. Create the Gateway using `create_gateway`.
3. Add Gateway Targets using `create_gateway_target`, to expose MCP tools that will be implemented in AWS Lambda or in existing RESTful services.

이 튜토리얼에서는 Amazon Cognito를 ID 공급자(IdP)로, AWS Lambda 함수를 대상으로, AWS IAM을 아웃바운드 인증으로 사용할 것입니다. 이 튜토리얼에서 시연된 동일한 개념은 다른 IdP나 다른 대상 유형을 사용할 때도 여전히 적용됩니다.

### Creating Amazon Cognito resources

이 튜토리얼에서는 다음 리소스를 이미 생성했고 해당 환경 변수를 설정했다고 가정합니다:

- IAM role for AWS Lambda execution (`gateway_lambda_iam_role`)
- AWS Lambda function for your simple math tools (`calc_lambda_arn`)
- AWS Lambda function for your restaurant reservation tool (`restaurant_lambda_arn`)
- Amazon Cognito user pools, giving you a client id (`cognito_client_id`) and a discovery URL (`cognito_discovery_url`)

Lets take a look at the JSON tool metadata for the restaurant API. Note that if we were integrating with existing REST services, the API specs would be provided using OpenAPI Schema instead.

In [ ]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

Here are the simple calculator APIs.

In [ ]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

Here is the AWS Lambda function implementation for the calculator tools.

In [ ]:
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
#### MCP 도구로 변환하려는 샘플 AWS Lambda 함수 생성calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda 함수가 다음 ARN으로 생성되었습니다: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda 함수 생성이 다음 메시지와 함께 실패했습니다: ",
            calc_lambda_resp["lambda_function_arn"],
        )

In [ ]:
calc_lambda_resp["lambda_function_arn"]

In [ ]:
#### MCP 도구로 변환하려는 샘플 AWS Lambda 함수 생성restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda 함수가 다음 ARN으로 생성되었습니다: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda 함수 생성이 다음 메시지와 함께 실패했습니다: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

In [ ]:
restaurant_lambda_resp["lambda_function_arn"]

In [ ]:
cognito_response = utils.setup_cognito_user_pool()

In [ ]:
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

In [ ]:
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

#### Let's create a few helper functions for using the control plane APIs

In [ ]:
def read_apispec(json_file_path):
    try:
        # read json file and return contents as string
        with open(json_file_path, "r") as file:
            # Parse JSON to Python object
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

#### Create Gateway helper function
Here is a helper function for creating an AgentCore Gateway given a name and
description. It uses Amazon Cognito as its IdP, and pulls the allowed client ID
and the discovery URL from environment variables, as those were already defined.
It also defaults to enabling semantic search on the resulting Gateway, and uses
a predefined IAM role.

In [ ]:
def create_gateway(gateway_name, gateway_desc):
    # Use Cognito for Inbound OAuth to our Gateway
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # Enable semantic search of tools
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # Create the gateway
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )
    # print(json.dumps(response, indent=2, default=str))
    return response["gatewayId"]

#### Create Gateway Target helper function
이 함수는 기존 Gateway에 새로운 AWS Lambda 대상을 생성합니다.
Simply provide the gateway ID, the name and description of the new target,
the ARN of the existing AWS Lambda function, and the JSON schema describing
the interfaces to the tools you want to expose from the gateway.

In [ ]:
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Add a Lambda target to the gateway
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # Use IAM as credential provider
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

### Creating your first AgentCore Gateway
Before we setup your first Gateway, let's take a quick look at how 
Gateway provides security, both for inbound requests to use MCP tools,
and outbound access from the Gateway to tools and resources.

![작동 방식](images/gateway_secure_access.png)

Now let's create the gateway for this tutorial, providing a name and a description.

In [ ]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"Gateway created with id: {gatewayId}.")

### Adding AgentCore Gateway Targets
이 튜토리얼에서는 이미 한 쌍의 Lambda 함수를 설치했다고 가정합니다. 하나는 간단한
math calculations, and another that simulates creating a restaurant reservation. We'll add a Gateway Target
for each of these functions.

Once we've added those targets, we'll add additional targets to simply drive higher MCP tool counts that
help us demonstrate the power of AgentCore Gateway search.

Now that the gateway is created, let's add a target for making restaurant reservations
via a Lambda function.

In [ ]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

Here we'll add a second target, this time with a Lambda that implements 4 basic tools (add, subtract,
곱셈, 나눗셈)과 투자 관리를 위한 75개의 생성된 도구 정의 세트(거래, 신용 연구, 정량 분석, 포트폴리오 관리)입니다. 투자 관리 도구 정의는 
actually implemented in the Lambda function. We are only adding them to demonstrate a large volume of tools.

In [ ]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"Calc Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

To demonstrate the power of gateway search, now we add a few more copies of the Calculator target, 
so that we end up with 300+ MCP tools exposed.

In [ ]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")

In [ ]:
add_more_tools(gatewayId=gatewayId)

In [ ]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

# Searching for tools from a Gateway

### Getting familiar with MCP list tools before we search

Let's define some utility functions to retrieve our MCP endpoint URL for a given Gateway ID, and to 
retrieve our JWT OAuth access token to securely use our Gateway.

In [ ]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

Now that our Gateway is created and has targets, let's grab the MCP URL to that
Gateway. We can retrieve the endpoint URL from the Gateway control plane based on the Gateway ID.

#### Using MCP Inspector against your Gateway

Now that we have an endpoint URL for the MCP server, and we have a JWT bearer token, you may want to explore
the MCP server with the MCP Inspector tool. MCP Inspector is an open source tool that can connect to any MCP
server, lets you list the tools provided, and even provides an easy to use tool invocation experience. 

From your terminal window, simply enter `npx @modelcontextprotocol/inspector` to launch the MCP Inspector. Then paste
in your Gateway endpoint URL and your JWT token to connect. Once connected, try out List Tools and Invoke Tool.

Here's a sample screenshot.

![MCP Inspector](images/mcp_inspector.png)

In [ ]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

MCP server security is based on OAuth. To interact with our Gateway, we'll need to
retrieve a JWT OAuth access token from our IdP.

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer token: {jwtToken}")

In [ ]:
# !npx @modelcontextprotocol/inspector

#### Creating helper functions that use jsonrpc to invoke MCP tools or list them
Let's define a helper function called `invoke_gateway_tool` that uses jsonrpc to invoke any of the
MCP tools exposed by an MCP Server, including of course, your Gateway. Given an endpoint URL and a JWT token,
you can use this utility to invoke any of the MCP tools that AgentCore Gateway made available
for you when you added Gateway Targets to your Gateway.

In [ ]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # print(f"Invoking tool {tool_params['name']}")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

Here's another utility function for using MCP's `tools/list` method for listing the MCP tools
available from your Gateway. Given a Gateway ID and a JWT Token, it retrieves the full set
해당 Gateway의 도구들을 가져와서 에이전트가 사용할 수 있는 형태의 목록을 반환합니다. 반환된 목록에는 
Strands Agents MCPAgentTool objects that are suitable for handing your Agent. 

Note that `tools/list` call is paginated, so the function needs to loop, getting a page of
한 번에 도구들을 가져오며, `nextCursor` 필드가 더 이상 채워지지 않을 때까지 계속합니다. 유틸리티 함수는 직접
HTTPS와 jsonrpc 프로토콜을 사용하여 엔드포인트를 호출합니다. 이는 도구를 나열하는 더 낮은 수준의 방법입니다
compared to the `MCPClient` class provided by Strands Agents. We'll see that experience later.

In [ ]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nListing tools for gateway {gateway_endpoint}")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

Lets use this helper function and see the results.

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\nFound {len(all_tools)} tools using jsonrpc to list MCP tools\n")

#### Using Strands Agents list_tools_sync() with pagination
If you have written any Python based MCP client, you are likely familiar with the `list_tools_sync()` method 
which returns the set of tools available from the MCP Server which the client is associated
with. But did you know MCP list tools is also paginated? By default, you will only get the first small
반환된 도구의 하위 집합입니다. 간단한 MCP 서버의 경우 이를 눈치채지 못했을 수도 있지만, 많은 실제 
MCP servers, your code needs to loop, grabbing pages of tools at a time
더 이상 남은 페이지가 없을 때까지 계속합니다. 다음 유틸리티 `get_all_mcp_tools_from_mcp_client`가 정확히 그 일을 합니다. 
It returns the full list of tools from a given Strands Agent MCP Client.

In [ ]:
def get_all_mcp_tools_from_mcp_client(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

Let's give it a try with our Gateway and find out how many tools the Python client finds. 
First we create an MCPClient object based on our endpoint URL and our JWT bearer token. Then 
we retrieve the full set of tools across many pages of tools returned by the MCP server.
Given the targets we added earlier, this should return 300+ tools.

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

We have now seen 3 different ways to get the full set of tools from your Gateway using it
as an MCP Server: 

1. directly using jsonrpc
2. using the `list_tools_sync()` method on the Strands Agent MCPClient
3. using the MCP Inspector tool (which uses jsonrpc behind the scenes). 

에이전트를 구축하는 일반적인 개발자의 경우 옵션 2를 사용하게 될 것입니다.

### Using the built-in Gateway semantic search tool
Now lets try our first semantic search on the Gateway using its built-in search tool provided as
an additional MCP tool that gets added to your MCP tool list.

First let's define a simple utility function to execute the search tool using MCP.
Just like for listing tools, we need the gateway endpoint and JWT token. Other than that,
우리가 전달해야 하는 것은 검색 쿼리뿐입니다. Gateway 검색 도구가 나머지를 처리할 것입니다.
matching that query against the serverless vector store that it automatically manages on your behalf.

In [ ]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [ ]:
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"tool search via direct Gateway invocation took {(end_time - start_time):.2f} seconds"
)
print(f"Top tool: {tools_found[0]['name']}")

대부분의 경우 1초 이내에 검색이 얼마나 빠르게 반환되는지 주목하세요. 결과가 반환됩니다
in descending order of search relevance based on matching the query to the tool metadata.
가장 관련성이 높은 도구가 목록의 맨 앞에 있습니다. 검색의 초기 구현은 다음을 반환합니다
up to 10 results. You could then use all of these tools in your agent, or simply pick a subset of
the most relevant matches.

# Using Strands Agents with an MCP server that has many tools

First, we select a model to use with our Strands Agent. 
이 노트북에서는 Amazon Bedrock 모델을 사용하고 있지만, Strands와 AgentCore는
can work with any LLM.

In [ ]:
bedrockmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

#### Simple Strands Agent using AgentCore Gateway for agent tools
Now lets show how easy it is to use a Strands Agent to leverage an MCP Server
AgentCore Gateway에서 제공됩니다. 우리의
simple example, we ask the agent to add some numbers.

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("add 100 plus 50 pass ")
    print(f"{result.message['content'][0]['text']}")

Strands Agents 프레임워크는 또한 에이전트 이벤트 루프를 우회하여 MCP 도구를 직접 호출할 수 있게 해줍니다.
Since Gateway tools are exposed as native MCP tools, this can be done against Gateway tools as well. Here
we call a Gateway MCP tool using the `agent.tool.<tool_name>(args)` syntax:

```python
direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
resp_json = json.loads(direct_result['content'][0]['text'])
```

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"direct result = {direct_result}")

In [ ]:
def get_search_tool(client):
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [ ]:
def search_using_strands(client, query):
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    # print(json.dumps(search_results, indent=4))
    return search_results

In [ ]:
def find_strands_tools(client, query, top_n):
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

# Adding tool search results to a Strands Agent

Now let's look at how the tools returned from a search can be added to a
Strands Agent. To make the coding simpler, let's provide a utility function that
maps tool search results to Strands MCPAgentTool objects. Simply pass in the 
search results, and indicate how many of those results you want to pass to your
agent.

In [ ]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")

In [ ]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("Searching for an ADDING tool from endpoint with full set of tools...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"Top tool found: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70")
    print(f"{result.message['content'][0]['text']}")

지연 시간 개선을 주목하세요. Gateway 검색에서 도구의 하위 집합을 사용하는 이 예제는 다음보다 훨씬 빠릅니다
agent invocation when depending on hundreds of tools.

# Showing 3x latency improvement by using tool search

Now that we know how to use Gateway MCP tools from a Strands agent, and we know how to search for tools and
add them to an agent, lets show the power of search. We'll highlight the significant latency reduction
and input token usage that can be delivered.

To demonstrate the latency and token reductions, we compare 2 approaches side by side:

1. **Without search**. We add the full set of MCP tools that the MCP server exposes (300+ in our case) to our agent and let the agent do its tool selection and invocation accordingly.
2. **검색 사용**. 두 번째 접근 방식에서는 당면한 주제를 기반으로 검색을 수행하고 가장 관련성이 높은 도구만 에이전트에 보냅니다. 요점을 증명하기 위해 두 가지 다른 주제를 사용합니다: 수학(숫자 더하기)과 음식(레스토랑 예약), 각각 다른 도구 세트가 필요합니다.

To normalize the latency distribution and get a meaningful comparison, we perform multiple iterations of
each approach. Also, to avoid overstating the gains, when doing the search approach, we include not only the
에이전트 호출의 지연 시간뿐만 아니라 도구 검색 수행의 지연 시간도 포함됩니다. 각 
iteration, we hand the agent two tasks: 

1. Math task -  add 2 numbers 
2. Food task - book a restaurant reservation

아래 결과는 이점을 보여주며, 3배의 지연 시간 감소와 더 큰 감소를 강조합니다
input token usage. Note that while token usage savings translate to cost savings, that may not be as impactful due to
the relatively lower cost of input tokens (for many model providers, input tokens are much lest costly). Even so, for 
large scale agent deployment, even input token usage costs can add up, so dynamic search can help reduce 
agent runtime costs as well.

#### Measure latency and token usage for agent using the entire set of MCP tools

In [ ]:
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

In [ ]:
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "add 100 plus <iteration>"
    food_input = (
        "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"
    )

    print("using agent with ALL tools...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\nTotal time: {full_elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")

#### Measure latency and token usage for agents using Gateway Search
Now we'll use a dynamic approach, calling search to find relevant tools, and then calling the
agent with only those relevant tools. Note that since we are resetting the agent on each 
conversation turn, we're also intializing the message list from conversation history of the prior turn.

In [ ]:
with client:
    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        print("Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        print(
            "Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

#### Compare results, higlighting benefits of search

In [ ]:
print(
    f"\n\nLatency without search: {full_elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s"
)
print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")

# Conclusion
이 튜토리얼에서는 Amazon Bedrock AgentCore Gateway와 그 내장된 
fully managed semantic search capability. You have seen the following:

- how to create a gateway with semantic search enabled
- how to add multiple gateway targets to surface 300+ MCP tools from a single endpoint
- how to list the tools on your gateway using 3 different approaches
- how to use the built-in semantic search tool to find relevant tools
- how to integrate search with your Strands Agent
- how to compare performance of an agent using a server with hundreds of tools versus one that uses semantic search to narrow tools to a specific topic

AgentCore Gateway search is helpful for more advanced use cases as well. By offering the search as a native
MCP tool and not just a control plane API, you can imagine giving your agents more autonomy to discover new
MCP servers, and find new capabilities at runtime leading to breakthroughs in solving more challenging problems.
또한 검색은 MCP 레지스트리와 에이전트 개발자를 지원하는 중요한 기반입니다 
design and build new agents.

# Cleaning up resources

First let's define some helper functions for cleaning up AgentCore Gateway resources.

In [ ]:
def delete_gatewaytarget(gateway_id):
    response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)

    print(f"Found {len(response['items'])} targets for the gateway")

    for target in response["items"]:
        print(
            f"Deleting target with Name: {target['name']} and Id: {target['targetId']}"
        )

        response = agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id, targetId=target["targetId"]
        )
        time.sleep(20)


def delete_gateway(gateway_id):
    response = agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)

### Deleting Gateway Targets

In [ ]:
delete_gatewaytarget(gateway_id=gatewayId)

### Deleting the Gateway itself

In [ ]:
delete_gateway(gateway_id=gatewayId)

In [ ]:
lambda_arns = [
    calc_lambda_resp["lambda_function_arn"],
    restaurant_lambda_resp["lambda_function_arn"],
]

for arn in lambda_arns:
    if utils.delete_gateway_lambda(arn):
        print(f"Deleted Lambda: {arn}")
    else:
        print(f"Lambda {arn} not found or deletion failed")

In [ ]:
# Gateway role cleanup
if utils.delete_gateway_iam_role():
    print("Gateway IAM role deleted")
else:
    print("Gateway IAM role not found or deletion failed")

# Cognito cleanup
if utils.delete_cognito_user_pool():
    print("Cognito pool deleted")
else:
    print("✗ Failed to delete Cognito pool")